In [30]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [31]:
random.uniform(0, 1)

0.8708776493948649

In [82]:
%%time
n_p = 2000
n_b = 40
l_0 = 1
l_p = 1
kappa = l_p/l_0

output = np.zeros([n_p*n_b, 3])

for poly in range(n_p):
    #first bead
    r1 = np.array([0, 0, 0])
    output[poly*n_b] = r1
    phi = 2*np.pi*random.uniform(0, 1)
    theta = np.arccos(random.uniform(-1, 1))
    u_2 = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
    r2 = u_2 * l_0
    output[poly*n_b + 1] = r2
    
    prev_r = r2
    for bead in range(n_b-2):
        r = random.uniform(0, 1)
        rho = (1/kappa)*np.log(np.exp(-kappa)+r*(np.exp(kappa)-np.exp(-kappa)))
        theta = np.arccos(rho)
        phi = 2*np.pi*random.uniform(0, 1)
        
        #define z' as previous r
        z_prime = output[poly*n_b + bead + 1]
        z_prime = z_prime / np.linalg.norm(z_prime)
        
        #define x' and y' using rotation matrix wrt phi about z axis (random phi angle)
        
        #rot_mat = np.array([[np.cos(phi), -np.sin(phi), 0], [np.sin(phi), np.cos(phi), 0], [0, 0, 1]])
        x_prime = np.random.randn(3)  # take a random vector (phi = random angle)
        x_prime -= x_prime.dot(z_prime) * z_prime      # make it orthogonal to z_prime
        x_prime /= np.linalg.norm(x_prime)
        
        y_prime = np.cross(z_prime, x_prime)
        
        #calculate new r_prime in x'y'z' axes
        
        r_prime = np.array([l_0 * np.sin(theta), 0, l_0 * np.cos(theta)])
        
        #covert r_prime from x'y'z' to xyz
        
        axes_1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        origin_1 = np.array([0, 0, 0])
        
        axes_2 = np.array([x_prime, y_prime, z_prime])
        origin_2 = output[poly*n_b + bead + 1]
        
        r_bead = np.dot(np.transpose(axes_1), (origin_2 - origin_1 + np.dot(axes_2, r_prime)))
        
        
        #u_bead = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
        #r_bead = u_bead * l_0
        
        output[poly*n_b + bead + 2] = output[poly*n_b + bead + 1] + r_bead #how to find next x y z coordinates
        
    #do all other beads- vectorize!

Wall time: 12 s


In [88]:
output[0:3,:]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.78257464,  0.09422281, -0.61538524],
       [ 1.99113823, -0.31663392, -0.48015203]])

In [89]:
import sys
sys.path.insert(1, '../wlcstat/')

In [90]:
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
import wlcstat.wlcave as wlcave
import wlcstat.wlcgreen as wlcgreen
import wlcstat.wlcstruc as wlcstruc
import wlcstat.poly_dyn as poly_dyn
import wlcstat.wlc_lcpoly as wlc_lcpoly
import wlcstat.poly_confine as poly_confine
import wlcstat.active_brown as active_brown
import wlcstat.util.wlc_vertex as vertex
import importlib

In [91]:
import importlib
importlib.reload(wlcave)
importlib.reload(wlcgreen)
importlib.reload(wlcstruc)
importlib.reload(poly_dyn)
importlib.reload(poly_confine)
importlib.reload(wlc_lcpoly)
importlib.reload(active_brown)

<module 'wlcstat.active_brown' from 'C:\\Users\\Thomas\\wlcsim_membrane\\wlcstat\\active_brown.py'>

In [92]:
n_p = 15
n_b = 200
n_b_calc = 200 
length_kuhn = 10
i_snap_0 = 9000
i_snap_f = 10000

k0 = 1e-2
kf = 1e2
nk = 50
k_vec = np.logspace(np.log10(k0), np.log10(kf), nk)

s2_sim = np.zeros(nk)

for i_snap in range(i_snap_0, i_snap_f + 1):
    u_snap = np.loadtxt('sim_data/u' + str(i_snap))
    r_snap = np.loadtxt('sim_data/r' + str(i_snap))
    for i_p in range(n_p):
        i_0 = n_b * i_p
        i_f = i_0 + n_b_calc
        u_i = u_snap[i_0:i_f, :]
        r_i = r_snap[i_0:i_f, :]
        
        #why can you separate by x, y, z?
        s_mat = np.exp(1j * np.outer(k_vec, r_i[:,0]) / 2)        
        s2_sim += np.abs(np.sum(s_mat, axis = 1)) ** 2 / (n_b_calc ** 2 * (i_snap_f - i_snap_0 + 1) * n_p * 3)
        s_mat = np.exp(1j * np.outer(k_vec, r_i[:,1]) / 2)        
        s2_sim += np.abs(np.sum(s_mat, axis = 1)) ** 2 / (n_b_calc ** 2 * (i_snap_f - i_snap_0 + 1) * n_p * 3)
        s_mat = np.exp(1j * np.outer(k_vec, r_i[:,2]) / 2)        
        s2_sim += np.abs(np.sum(s_mat, axis = 1)) ** 2 / (n_b_calc ** 2 * (i_snap_f - i_snap_0 + 1) * n_p * 3)

OSError: sim_data/u9000 not found.

In [ ]:
plt.figure(figsize=(10,8))
font = {'family' : 'serif',
        'weight':'normal',
        'size': 18}
plt.rc('font', **font)
    
plt.plot(k_vec, 1-s2_sim,'o')
#plt.plot(k_vec, 1-s2_th,'-')    
#plt.plot(k_vec, 1-s2_asymp,'--')    

plt.xscale('log')
plt.yscale('log')

plt.xlabel(r'$K$')
plt.ylabel(r'Structure Factor $S_{2}(K;N)$')
plt.xlim((1e-2, 1e2))
plt.tight_layout()
plt.show()